# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [28]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

In [29]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [30]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [31]:
# Create a Beautiful Soup Object
html = browser.html
page_soup = soup(html, 'html.parser')


In [32]:
# Extract all rows of data
data_rows = page_soup.find_all('tr')


### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [33]:
# Create an empty list
rows = []
# Loop through the scraped data to create a list of rows
for row in data_rows[1:]:
    columns = row.find_all('td')
    row_data = [column.text.strip() for column in columns]
    rows.append(row_data)

In [34]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
column_names = ['id', 'terrestrial_date', 'sol', 'ls', 'month', 'min_temp', 'pressure']
data = pd.DataFrame(rows, columns=column_names)


In [35]:
# Confirm DataFrame was created successfully
print(data)

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [36]:
# Examine data type of each column
print(data.dtypes)

In [37]:
# Change data types for data analysis
data['terrestrial_date'] = pd.to_datetime(data['terrestrial_date'])
data['sol'] = data['sol'].astype(int)
data['ls'] = data['ls'].astype(float)
data['min_temp'] = data['min_temp'].astype(float)
data['pressure'] = data['pressure'].astype(float)

In [38]:
# Confirm type changes were successful by examining data types again
print(data.dtypes)

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [39]:
# 1. How many months are there on Mars?
months_count = data['month'].nunique()
print("Number of months on Mars:", months_count)

In [40]:
# 2. How many Martian days' worth of data are there?
martian_days_count = data['sol'].nunique()
print("Number of Martian days worth of data:", martian_days_count)

In [41]:
# 3. What is the average low temperature by month?
average_temperatures = data.groupby('month')['min_temp'].mean()


In [42]:
# Plot the average temperature by month
average_temperatures.plot(kind='bar')
plt.xlabel('Month')
plt.ylabel('Average Minimum Temperature (Celsius)')
plt.title('Average Minimum Temperature by Month')
plt.show()

In [43]:
# Identify the coldest and hottest months in Curiosity's location
coldest_month = average_temperatures.idxmin()
hottest_month = average_temperatures.idxmax()
print("Coldest month:", coldest_month)
print("Hottest month:", hottest_month)

In [44]:
# 4. Average pressure by Martian month
average_pressure = data.groupby('month')['pressure'].mean()

In [45]:
# Plot the average pressure by month
average_pressure.plot(kind='bar')
plt.xlabel('Month')
plt.ylabel('Average Pressure')
plt.title('Average Pressure by Month')
plt.show()

In [46]:
# 5. How many terrestrial (earth) days are there in a Martian year?
terrestrial_days_in_martian_year = data['terrestrial_date'].nunique()
print("Number of terrestrial days in a Martian year:", terrestrial_days_in_martian_year)


On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [47]:
# Write the data to a CSV
data.to_csv('mars_weather_data.csv', index=False)

In [48]:
browser.quit()